# Segundo parcial (Aprendizaje por refuerzo)
 <h3>Implementacion aprendizaje por refuerzo, para que un agente pueda jugar cuatro en raya.<h3>
 <HR>
 <h3>
  NOMBRES:

  * POLO ORELLANA BRAYAN SIMON
  <br>
  CARRERA: INGENIERIA DE SISTEMAS
  <BR>
  FECHA: 04/06/2024 <BR>

  * [Enlace al colab](https://drive.google.com/file/d/1uTXm41W4uwyoAEMgwZDGQg8ZJWQB1Nli/view?usp=sharing)
  
  * [Enlace al git hub](https://github.com/bspoloo/SIS420-012024/tree/main/Parciales/Segundo%20Parcial/Aprendizaje%20por%20refuerzo)

 <h3>

Para resolver este juego con axr, en primer lugar definimos una tabla de números, uno por cada posible estado del juego. Cada numero respresentará la probabilidad de ganar el juego desde ese estado, el *valor* del estado. Así pues, la tabla sería la *función de valor*. Un estado $A$ es considerado mejor que un estado $B$ si el valor estimado de la probabilidad de ganar el juego desde $A$ es mayor que desde $B$. Si jugásemos con las Xs, todos los estados con Cuatro X en raya tenría un valor de 1, ya que hemos ganado el juego. De la misma manera, cualquier estado con tres Os en raya tendría un valor de 0, hemos perdido. Para la inicialización de la tabla, podemos establecer el resto de valores en 0.5 (50% de posibilidades de ganar).


* 0 (vacío)
* 1 (jugador 1)
* -1 (jugador 2)

## Board
Este sera nuestro tablero de juego, donde este incializara un tablero 4x4 en ceros.

validara los movimientos disponibles, esto con una lista que contendra los espacios vacios.



In [8]:
import numpy as np
class Board():
    def __init__(self):
        self.state = np.zeros((4,4)) 

    def valid_moves(self):
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0] #Devuelve una lista de todas las posiciones disponibles (vacías) en el tablero donde se pueden realizar movimientos.

    def update(self, symbol, row, col): #Actualiza el tablero con el movimiento de un jugador.
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError ("movimiento ilegal !")

    def is_game_over(self): #Comprueba el estado actual del juego para determinar si hay un ganador, un empate o si el juego debe continuar.
        # comprobar filas y columnas
        if (self.state.sum(axis=0) == 4).sum() >= 1 or (self.state.sum(axis=1) == 4).sum() >= 1:
            return 1
        if (self.state.sum(axis=0) == -4).sum() >= 1 or (self.state.sum(axis=1) == -4).sum() >= 1:
            return -1 
        # comprobar diagonales
        diag_sums = [
            sum([self.state[i, i] for i in range(4)]),
            sum([self.state[i, 4 - i - 1] for i in range(4)]),
        ]
        if diag_sums[0] == 4 or diag_sums[1] == 4:
            return 1
        if diag_sums[0] == -4 or diag_sums[1] == -4:
            return -1        
        # empate
        if len(self.valid_moves()) == 0:
            return 0
        # seguir jugando
        return None

    def reset(self):# Reinicia el tablero.
        self.state = np.zeros((4,4))


## Game
En esta parte se creara la clase Game que hara la parte de exploracion de un agente, tomando en cuenta dos agentes, con una probabilidad de exploracion de 50%, esto hara que incialmente escoja un estado al azar al momento de jugar el juego.

In [15]:
from tqdm import tqdm 

class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board()

    def selfplay(self, rounds=100):
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)): #Utiliza tqdm para mostrar una barra de progreso durante las partidas.
            self.board.reset() #Reinicia el tablero antes de cada ronda.
            for player in self.players: #Reinicia el estado de cada jugador antes de cada ronda.
                player.reset()
            game_over = False #Inicializa la variable game_over en False.
            while not game_over: #Mientras el juego no haya terminado, los jugadores realizan movimientos.
                for player in self.players:
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players: #Actualiza el estado de cada jugador después de cada movimiento.
                        player.update(self.board)
                    if self.board.is_game_over() is not None: #Comprueba si el juego ha terminado después de cada movimiento.
                        game_over = True
                        break
            self.reward() #Calcula la recompensa para cada jugador después de cada ronda.
            for ix, player in enumerate(self.players): #Actualiza las estadísticas de victorias para cada jugador después de cada ronda.
                if self.board.is_game_over() == player.symbol:  # si el jugador gana
                    wins[ix] += 1
        return wins


    def reward(self):
        winner = self.board.is_game_over() #Determina el ganador de la partida.
        if winner == 0: # empate
            for player in self.players:
                player.reward(0.5)
        else: # le damos 1 recompensa al jugador que gana
            for player in self.players: #Calcula la recompensa para cada jugador.
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

## Agente
Esta clase tiene una taza de aprendizaje y una probabilidad de exploracion del 50%, cuenta con las funciones de resetar sus pocisiones, moverse, actualizar y su premio que actualiza la funcion del valor de acuerdo a si la pocision actual no esta en la tabla: usando la siguiente formula para la funcion del valor:

Mientras el agente va jugando, tendremos que actualizar la función de valor. Para ello, después de cada movimiento, cambiaremos el valor del estado del que venimos para que se acerque al valor del estado actual.

$$
V(S_t) \leftarrow V(S_t) + \alpha [V(S_{t+1}) - V(S_t)]
$$
donde $S_t$ denote el estado del que venimos, $S_{t+1}$ es el nuevo estado después del movimiento, $V(S_t)$ es el valor del estado $S_t$ y $\alpha$ es el ratio de aprendizaje.

In [17]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5): #Inicializa el agente con un valor de aprendizaje alpha y una probabilidad de exploración prob_exp.
        self.value_function = {} # tabla con pares estado -> valor
        self.alpha = alpha         # learning rate
        self.positions = []       # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp   # probabilidad de explorar

    def reset(self): #Reinicia el estado del agente antes de cada ronda.
        self.positions = []

    def move(self, board, explore=True): #Determina el movimiento del agente en función del tablero actual.
        valid_moves = board.valid_moves()
        # exploracion
        if explore and np.random.uniform(0, 1) < self.prob_exp: #si el valor aleatorio es menor que la probabilidad de exploración, el agente realiza un movimiento aleatorio.
            # vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves)) #Selecciona una posición aleatoria de la lista de movimientos válidos.
            return valid_moves[ix]
        # explotacion
        # vamos a la posición con más valor
        max_value = -1000 #Inicializa el valor máximo en -1000. esto para ir aumentando con el tiempo
        for row, col in valid_moves: #Itera sobre todas las posiciones válidas en el tablero.
            next_board = board.state.copy() #Copia el tablero actual.
            next_board[row, col] = self.symbol #Realiza el movimiento en la posición actual.
            next_state = str(next_board.reshape(4*4)) #Convierte el tablero actual en una cadena de texto.
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state) #Obtiene el valor de la cadena de texto actual.
            if value >= max_value: #Si el valor actual es mayor o igual al valor máximo, actualiza el valor máximo y la mejor posición.
                max_value = value 
                best_row, best_col = row, col
        return best_row, best_col #Devuelve la mejor posición.

    def update(self, board): #Actualiza el estado del agente después de cada movimiento.
        self.positions.append(str(board.state.reshape(4*4)))

    
    def reward(self, reward):
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por tods los estados actualizando su valor en la tabla
        for p in reversed(self.positions): #Itera sobre todas las posiciones en orden inverso.
            if self.value_function.get(p) is None: #Si la posición actual no está en la tabla, inicializa su valor en 0.
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p]) #Actualiza el valor de la posición actual en función de la recompensa recibida.
            reward = self.value_function[p] #Actualiza la recompensa con el valor actual.

### Exploracion del agente

* si en un 3x3 = 9, era !9 = 362880 estados posibles,  entonces
* en un 4x4 = 16, entonces !16 = 20922789888000 estados posibles lo cual es imposible de entrenar, ya que nos dara un tiempo de ejecución altisimo.

por lo tanto solo se probrara con un maximo de 1000000 rondas

La línea ``game.selfplay(1000000)`` inicia un proceso de autojuego donde los dos agentes juegan entre sí un millón de veces. Durante este proceso, los agentes aprenden de sus experiencias, mejorando sus políticas de acción a medida que juegan más y más juegos:

In [20]:
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)


game.selfplay(1000000)

100%|██████████| 1000000/1000000 [2:50:54<00:00, 97.52it/s]  


[352115, 241550]

In [21]:
import pandas as pd

funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True) #Ordena la tabla de valores en función de los valores de los estados.
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]}) #Crea un DataFrame con los estados y los valores ordenados.

tabla

,estado,valor
0,[ 0. 0. -1. 1. 0. 0. 0. 1. 0. 0. -1. ...,1.0
1,[ 1. 0. 0. 0. 1. -1. 0. -1. 1. 0. 0. ...,1.0
2,[ 1. 0. 0. -1. 0. 1. 0. 0. 0. -1. 1. -...,1.0
3,[ 1. 0. 0. 0. 0. 1. 0. 0. -1. -1. 1. -...,1.0
4,[ 0. 1. 0. -1. 0. 1. 0. -1. -1. 1. 0. ...,1.0
...,...,...
2266779,[-1. 0. 0. -1. 0. 1. -1. 1. -1. 0. 1. ...,0.0
2266780,[-1. 0. 0. -1. 0. 0. -1. 1. -1. 0. 1. ...,0.0
2266781,[ 0. 0. 0. -1. 0. 0. -1. 1. -1. 0. 1. ...,0.0
2266782,[ 0. 0. 0. -1. 0. 0. -1. 0. -1. 0. 1. ...,0.0


In [22]:
import pickle

with open('agente4x4.pickle', 'wb') as handle: #Guarda la tabla de valores en un archivo pickle.
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
import pandas as pd

# Convertir el diccionario en un DataFrame
df = pd.DataFrame(list(agent1.value_function.items()), columns=['State', 'Value']) 

# Guardar el DataFrame en un archivo .csv
df.to_csv('agente_4x4.csv', index=False)